<a href="https://colab.research.google.com/github/bhouldridge/repertoire/blob/main/Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 3

By: Benjamin Houldridge

August 10th 2020

##Problem Definition

This is a regression problem to determin the revenue of a movie given many datapoints.

##Data Collection/Sources



###Imports


In [ ]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
sns.set
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import graphviz
import pydotplus
from IPython.display import display
from sklearn import tree

###Import Data

In [ ]:
original_data_link = '/content/drive/My Drive/Exercises - Benjamin/Project 3/Copy of Project_3_Movie_Data.csv'

In [ ]:
original_data = pd.read_csv(original_data_link)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Data Cleaning

###Check Data

In [ ]:
original_data.head()


In [ ]:
original_data.tail()

In [ ]:
original_data.info()

In [ ]:
original_data.shape

In [ ]:
data = original_data.copy()

###Fill Nan Values

####Homepage

In [ ]:
data['homepage'].fillna(0, inplace=True)

In [ ]:
homepage = pd.Series(np.zeros(data.shape[0]))
for idx, value in enumerate(data['homepage']):
  if type(value) == str:
    homepage[idx] = 1
data['homepage'] = homepage

####ID

In [ ]:
data.drop('id', axis=1, inplace=True)

####Overview

Little to no useful information in this column.

In [ ]:
data.drop('overview', axis=1, inplace=True)

####Tagline

In [ ]:
data['tagline'].fillna(0, inplace=True)

In [ ]:
tagline = pd.Series(np.zeros(data.shape[0]))
for idx, value in enumerate(data['tagline']):
  if type(value) == str:
    tagline[idx] = 1
data['tagline'] = tagline

####Runtime

In [ ]:
data[data['runtime'].isna()]

In the runtime column two entries are Nan. These also have a revenue of 0, so I'm assuming These movies have incompleet information. These entries will be deleted.

In [ ]:
data.drop([182,527], inplace=True)

In [ ]:
data.reset_index(inplace=True, drop=True)

In [ ]:
data.info()

###Abnormal Columns

####Status

In [ ]:
data['status'].unique()

In [ ]:
len(data[data['status']!='Released'])

There are only seven entries that are rumored or post production and so we delet those rows so they don't interfeer with our predition.

In [ ]:
data = data[data['status']=='Released']

In [ ]:
data.drop('status', axis=1, inplace=True)

In [ ]:
data.reset_index(inplace=True, drop=True)

####Title

In [ ]:
data.shape

In [ ]:
len(data['title.1'].unique())

In [ ]:
len(data['title'].unique())

In [ ]:
len(data['original_title'].unique())

Find rows with duplicate titles.

In [ ]:
titles = []
for idx, title in enumerate(data['title']):
  if title in titles:
    print(title + ' at ' + str(idx))
  else:
    titles.append(title)

In [ ]:
for idx, title in enumerate(data['title']):
  if title == 'Out of the Blue' or title == 'The Host':
    print(title + ' at ' + str(idx))

In [ ]:
data.loc[[725,847,1073,2785]]

There are movies with the same title but they are clearly difforent movies. the two titel columns don't provide any clasification or value based data beyond this so they will be deleted.

In [ ]:
data.drop(['title','title.1', 'original_title'], axis=1, inplace=True)

####Movie ID

In [ ]:
len(data['movie_id'].unique())

All movie id's are unique so no usefull data is found here.

In [ ]:
data.drop('movie_id', axis=1, inplace=True)

In [ ]:
data.info()

###Columns Unraveled

####Functions

In [ ]:
def decode(column):
  """Return a Pandas Series with a unloaded json object for each row."""

  new_column = pd.Series(np.zeros(data.shape[0]))
  for idx, cell in enumerate(data[column]):
    new_column[idx] = json.loads(cell)
  return new_column

In [ ]:
def decode2(column):
  """Return a list with a unloaded json object for each element."""

  column_decoded = []
  for cell in data[column]:
    column_decoded.append(json.loads(cell))
  return column_decoded

In [ ]:
def pull_items(column, item):
  """Return Pandas Series of lists for all items found in each decoded row."""
  
  new_column = pd.Series(np.zeros(len(column)))
  for idx, cell in enumerate(column):
    items = [] 
    for subcell in cell:
      items.append(subcell[item])
    new_column[idx] = items
  return new_column

In [ ]:
def set_scores(column, item):
  """Return a dictionary with all items as keys and number of times found as values."""

  output = {}
  for cell in column:
    for subcell in cell:
      if subcell[item] in output:
        output[subcell[item]] += 1
      else:
        output[subcell[item]] = 1
  return output

In [ ]:
def set_scores2(decoded_list, item):
  """Return a dictionary with all items as keys and number of times found as values."""

  empty_list = []
  column_scored = {}
  for cell in decoded_list:
    if cell == empty_list:
      continue
    else:
      for subcell in cell:
        if item in subcell:
          if subcell[item] in column_scored:
            column_scored[subcell[item]] += 1
          else:
            column_scored[subcell[item]] = 1
  return column_scored

In [ ]:
def set_scores3(decoded_list, job_title):
  """Retuen a dictionary with names and keys and number of occurences as values"""
  
  column_scored = {}
  for cell in crew_decoded:
    for subcell in cell:
      if subcell['job'] == job_title:
        if subcell['name'] in column_scored:
          column_scored[subcell['name']] += 1
        else:
          column_scored[subcell['name']] = 1
  return column_scored

In [ ]:
def score_column(column, item, score_dictonary, limit=100):
  """Returns a Pandas Series of summed up individual subcell scores for each row."""

  new_column = pd.Series(np.zeros(data.shape[0]))
  for idx, cell in enumerate(column):
    row_score = 0
    for subcell in cell[:limit]:
      row_score = score_dictonary[subcell[item]]
    new_column[idx] = row_score
  return new_column

In [ ]:
def score_column2(decoded_list, item, score_dictonary, limit=100):
  """Returns a Pandas Series of summed up individual subcell scores for each row."""

  empty_list = []
  new_column = pd.Series(np.zeros(len(decoded_list)))
  for idx, cell in enumerate(decoded_list):
    if cell == empty_list:
      new_column[idx] = 0
    else:
      score = 0
      for subcell in cell[:limit]:
        if item in subcell:
          score += score_dictonary[subcell[item]]
      new_column[idx] = score
  return new_column

In [ ]:
def score_column3(decoded_list, job_title, score_dictonary, limit=100):
  """Returns a Pandas Series of summed up individual subcell scores for each row."""

  empty_list = []
  new_column = pd.Series(np.zeros(len(decoded_list)))
  for idx, cell in enumerate(decoded_list):
    if cell == empty_list:
      new_column[idx] = 0
    else:
      scores = []
      for subcell in cell:
        if subcell['job'] == job_title:
          scores.append(score_dictonary[subcell['name']])
      new_column[idx] = sum(sorted(scores, reverse=True)[:limit])
  return new_column

In [ ]:
data.head()

####Genres

In [ ]:
data['genres']

In [ ]:
genres_unpacked = decode('genres')

In [ ]:
data['genres_clean'] = pull_items(genres_unpacked, 'name')

In [ ]:
data.drop('genres', axis=1, inplace=True)

####Keywords

In [ ]:
data['keywords']

In [ ]:
keywords_decoded = decode('keywords')

In [ ]:
keyword_frequency = set_scores(keywords_decoded, 'name')

In [ ]:
data['scored_keywords'] = score_column(keywords_decoded, 'name', keyword_frequency)

In [ ]:
data.drop('keywords', axis=1, inplace=True)

####Production Companies

In [ ]:
data['production_companies']

The empty lists in the data are not playing nice with numpy, so decode2 is written to deal with that by using a list insted of a numpy array, and pandas series.

In [ ]:
production_companies_decoded = decode2('production_companies')

Create a scoring dictionary for Production Companies

In [ ]:
company_scores = set_scores2(production_companies_decoded, 'name')

Give a score to production companies and create a new column for it in the data.

In [ ]:
data['scored_production_companies'] = score_column2(production_companies_decoded, 'name', company_scores)

Delet the production_companies column

In [ ]:
data.drop('production_companies', axis=1, inplace=True)

####Production Countries

In [ ]:
data['production_countries']

create scoring dictionary, score the column, and replace new scored column with the old one.

In [ ]:
production_countries_decoded = decode2('production_countries')

In [ ]:
country_scores = set_scores2(production_countries_decoded, 'name')

In [ ]:
data['scored_production_countries'] = score_column2(production_countries_decoded, 'name', country_scores)

In [ ]:
data.drop('production_countries', axis=1, inplace=True)

####Release Date

In [ ]:
data['release_date']

Seperate into a release year column and a release quarter column.

In [ ]:
int(data['release_date'][0][5:7])

In [ ]:
release_year = pd.Series(np.zeros(data.shape[0]))
release_quarter = pd.Series(np.zeros(data.shape[0]))
for idx, value in enumerate(data['release_date']):
  release_year[idx] = int(value[:4])
  if int(value[5:7]) <= 3:
    release_quarter[idx] = 'q1'
  elif int(value[5:7]) <= 6:
    release_quarter[idx] = 'q2'
  elif int(value[5:7]) <= 9:
    release_quarter[idx] = 'q3'
  else:
    release_quarter[idx] = 'q3'
release_quarter

In [ ]:
data['release_year'] = release_year
data['release_quarter'] = release_quarter

In [ ]:
data.drop('release_date', axis=1, inplace=True)

####Spoken Languages

In [ ]:
data['spoken_languages']

In [ ]:
spoken_languages_decoded = decode('spoken_languages')

In [ ]:
data['spoken_languages_clean'] = pull_items(spoken_languages_decoded, 'name')

In [ ]:
data.drop('spoken_languages', axis=1, inplace=True)

####Cast

In [ ]:
data['cast']

In [ ]:
data['cast'][0]

In [ ]:
cast_decoded = decode2('cast')

In [ ]:
cast_scores = set_scores2(cast_decoded, 'name')

In [ ]:
cast_scores

In [ ]:
data['scored_cast'] = score_column2(cast_decoded, 'name', cast_scores, 3)

In [ ]:
data.drop('cast', axis=1, inplace=True)

####Crew

In [ ]:
data['crew']

In [ ]:
data['crew'][0]

In [ ]:
crew_decoded = decode2('crew')

In [ ]:
jobs = {}
for crew in crew_decoded:
  for member in crew:
    if member['job'] in jobs:
      jobs[member['job']] += 1
    else:
      jobs[member['job']] = 1

In [ ]:
len(jobs)

In [ ]:
sorted_jobs = sorted(jobs.items(), key=lambda x: x[1], reverse=True)
print(sorted_jobs[:5])

#####Producer

In [ ]:
score_producers = set_scores3(crew_decoded, 'Producer')
data['scored_producers'] = score_column3(crew_decoded, 'Producer', score_producers)

In [ ]:
score_producers

#####Executive Producer

In [ ]:
score_executive_producer = set_scores3(crew_decoded, 'Executive Producer')
data['scored_executive_producers'] = score_column3(crew_decoded, 'Executive Producer', score_executive_producer)

#####Director

In [ ]:
score_directors = set_scores3(crew_decoded, 'Director')
data['scored_director'] = score_column3(crew_decoded, 'Director', score_directors)

In [ ]:
score_directors

#####Screenplay

In [ ]:
score_screenplay = set_scores2(crew_decoded, 'Screenplay')
data['scored_screenplay'] = score_column2(crew_decoded, 'Screenplay', score_screenplay)
data['scored_screenplay'].unique()
score_screenplay

In [ ]:
data.drop('crew', axis=1, inplace=True)

In [ ]:
data.info()

In [ ]:
data_clean = data.copy()

##Exploratory Data Analysis

###Functions

In [ ]:
def root_mean_squared_percentage_error(true_values, predicted_values):
  """Return the root mean squared percentage error"""

  rmspe = np.sqrt(np.mean(np.square((true_values - predicted_values)/true_values)))*100
  return rmspe

In [ ]:
def cross_validation(training_data, outcomes_data, model, itrs=100):
  """returns the average rmspe of the data put throught the model itr times."""

  errors = np.zeros(itrs)
  for idx in range(0, itrs):
    x_train, x_test, y_train, y_test = train_test_split(training_data, 
                                                        outcomes_data, 
                                                        test_size=.15)
    model.fit(x_train, y_train)
    y_predict = model.predict(x_test)
    errors[idx] = np.sqrt(mean_squared_error(y_test, y_predict))
  return errors.mean()

In [ ]:
def one_hot_lists(column):
  """Return a dataframe with one hot encoding of the input column."""

  unique_values = []
  for row in column:
    for value in row:
      if value not in unique_values:
        unique_values.append(value)
  new_dataframe = pd.DataFrame(np.zeros([len(column), len(unique_values)]), columns=unique_values)
  for column_name in new_dataframe.columns:
    for idx in range(len(column)):
      if column_name in column[idx]:
        new_dataframe[column_name][idx] = 1
      
  return new_dataframe

###Seperate Numeric and Catagorical

In [ ]:
outcomes = data_clean['revenue'].copy()
numerical_data = data_clean.select_dtypes(exclude='object').drop('revenue', axis=1).copy()
catagorical_data = data_clean.select_dtypes(include='object').copy()

###Scale Numerical Data



In [ ]:
scaler = preprocessing.StandardScaler()
numerical_data_scaled = pd.DataFrame(scaler.fit_transform(numerical_data), columns=numerical_data.columns)
numerical_data_scaled['Ones'] = 1
numerical_data_scaled.shape

###One-Hot Catigorical Features

In [ ]:
catagorical_data
print(catagorical_data.shape)

In [ ]:
catagorical_data = catagorical_data.join(one_hot_lists(catagorical_data['genres_clean']))
print(catagorical_data.shape)

In [ ]:
catagorical_data.drop('genres_clean', axis=1, inplace=True)
print(catagorical_data.shape)

In [ ]:
catagorical_data = catagorical_data.join(one_hot_lists(catagorical_data['spoken_languages_clean']))
print(catagorical_data.shape)

In [ ]:
catagorical_data.drop('spoken_languages_clean', axis=1, inplace=True)
print(catagorical_data.shape)

In [ ]:
catagorical_data = catagorical_data.join(pd.get_dummies(catagorical_data['original_language']))
print(catagorical_data.shape)

In [ ]:
catagorical_data.drop('original_language', axis=1, inplace=True)
print(catagorical_data.shape)

In [ ]:
catagorical_data = catagorical_data.join(pd.get_dummies(catagorical_data['release_quarter']))
print(catagorical_data.shape)

In [ ]:
catagorical_data.drop('release_quarter', axis=1, inplace=True)

In [ ]:
print(catagorical_data.shape)
catagorical_data.head()

###Combine Data

In [ ]:
training_data = numerical_data_scaled.join(catagorical_data).copy()

In [ ]:
training_data.shape

###Decision Tree

In [ ]:
model = DecisionTreeRegressor()

In [ ]:
cross_validation(training_data, outcomes, model)

Errors on the order of x10^8 is not good. Could be over or under fitting. Lets try a few difforent tree depths to see what hapens to the error.

In [ ]:
tree_depth_error =[]
for depth in range(1,11):
  model = DecisionTreeRegressor(max_depth=depth)
  error = cross_validation(training_data, outcomes, model)
  tree_depth_error.append(error)
  print('For a tree of depth {}, the cross validation error is {}.'.format(
      depth, error))

Looks as though 5 is the best depth but the error is still huge for all depths.

###Random Forest

In [ ]:
model = RandomForestRegressor(n_estimators=10)

In [ ]:
cross_validation(training_data, outcomes, model)

About an order of magnitude better results than with a single tree.

In [ ]:
random_tree_errors = []
for trees in range(1, 10):
  model = RandomForestRegressor(n_estimators=trees)
  error = cross_validation(training_data, outcomes, model, itrs=50)
  random_tree_errors.append(error)
  print('For {} trees, the cross validation error is {}.'.format(
      trees, error))

Looks as though the first few additions of additional trees realy helps the predictions but progress stops after 5 trees.

###XGBoost

In [ ]:
model = xgb.XGBRegressor(verbosity=1, objective='reg:squarederror')

In [ ]:
cross_validation(training_data, outcomes, model, itrs=50)

In [ ]:
xgb_rate_error = []
for rate in [.1,.2,.3,.4,.5,.6,.7,.8,.9]:
  model = xgb.XGBRegressor(learning_rate=rate, verbosity=1, objective='reg:squarederror')
  error = cross_validation(training_data, outcomes, model, itrs=20)
  xgb_rate_error.append(error)
  print('For a {} learning rate, the cross validation error is {}.'.format(
      rate, error))

A learing rate of around .2  yealds the best result.

In [ ]:
xgb_tree_error = []
for trees in range(1, 102, 10):
  model =  xgb.XGBRegressor(learning_rate=.2, n_estimators=trees, verbosity=1, objective='reg:squarederror')
  error = cross_validation(training_data, outcomes, model, itrs=50)
  xgb_tree_error.append(error)
  print('For {} trees, the cross validation error is {}.'.format(
      trees, error))

After just 20 trees or so the benifits seem neglegable.

##Data Processing

The final model will be an XGBoost Regressor with a learning rate of .2, and 30 trees.

In [ ]:
model =  xgb.XGBRegressor(learning_rate=.2, n_estimators=30, verbosity=1, objective='reg:squarederror')

In [ ]:
cross_validation(training_data, outcomes, model)

##Data Visualization

####Visualize Tree

In [ ]:
model = DecisionTreeRegressor(max_depth=3)
model.fit(training_data, outcomes)
dot_data = tree.export_graphviz(model, feature_names=training_data.columns)
display(graphviz.Source(tree.export_graphviz(model, feature_names=training_data.columns)))
pydot_graph = pydotplus.graph_from_dot_data(dot_data)
pydot_graph.write_png('original_tree.png')
pydot_graph.set_size('"5,5!"')
pydot_graph.write_png('resized_tree.png')

####XGBoost Feature scores

In [ ]:
imoprtances = model.feature_importances_
most_important = zip(imoprtances, training_data.columns)
most_important = sorted(most_important, reverse=True)
most_important[0][1]
scores = []
important_features = []

for num in range(10):
  scores.append(most_important[num][0])
  important_features.append(most_important[num][1])
print(scores, important_features)

In [ ]:
fig, ax = plt.subplots();
y_pos = np.arange(len(scores));

ax.barh(y_pos, scores);
ax.set_yticks(y_pos);
ax.set_yticklabels(important_features);
ax.invert_yaxis();  # labels read top-to-bottom
ax.set_xlabel('Importance');
ax.set_title('Feature Importance');
ax.set
plt.show(block=False);